# Generative AI

One of the major breakthroughs in AI in the past decade is in the area of _generative_ AI models. Large language models (LLMs) such as ChatGPT - by some considered synonimous to "AI" - are in fact generative models.These models are currently able to generate text, images, and video, to an increasingly accurate and realistic level. Intriguingly, it is not yet clear to what extent the generation of for example text implies "understanding" of the topics on the part of the network. Although the network architecture, training procedure, and data input strategy are developed and known by the human designers, our insight into how trained generative networks actually perform the task is still very limited.

This notebook is a brief introduction to generative AI models and some of the basic techniques. It draws from the material in “Deep learning: Foundations and concepts”, by Bishop, C. M., & Bishop, H. (2023), and other original works from the literature.

## Discriminatory vs. generative models

Initial models treated in courses on machine learning for AI typically involve _discriminatory_ models. An example is an artificial feedforward neural network that has to learn a function $y = f(\mathbf{x})$ with supervised learning, based on training pairs $(\mathbf{x}_i, {y}^{*}_i)$. Such models essentially learn the probability distribution $P(y|x)$.

Generative models, on the other hand, learn to generate new samples $\mathbf{x}$ that resemble the training data $\{\mathbf{x}_i\}$. In other words, they try to represent $P(x)$. The aim is very similar to modeling in science, i.e., the aim for generative models is to represent the underlying processes that give rise to the data. Eventually, modelling these processes will also useful for predicting variables of interest. One can think of how weather forecasts usually depend on various weather models, and how it would compare to purely discriminatory models, trained with samples of measurements and targets (like predicted temperature or other weather conditions).

The desired benefits of generative models are:

* They can capture causal relationships that will generalize better to unseen situations.
* They lead to useful intermediate representations that can serve downstream (discriminatory) tasks.
* Since they do not need labels, they expand the data on which can be learned. The size of the training data is very important for the performance of deep neural networks.
* Some models permit for the use of domain knowledge to constrain the generation process, improving their accuracy and generalization capability.

Generative AI has been studied since at least the early 1980s, e.g., with “Boltzmann” machines, but it has blossomed with the arrival of deep learning.

We will treat the following three approaches:
- Variational Auto-Encoders (VAEs)
- Generative Adversarial Networks (GANs)
- Diffusion models

## Variational Auto-encoder



